In [1]:
import lxml
import requests
import pandas as pd
import time
from bs4 import BeautifulSoup
import csv

SACAR LISTA JUGADORES

Diccionario con jugadores liga española mas valiosos

In [4]:
# Cabeceras HTTP para simular una solicitud desde un navegador
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36'
}

# URL base sin el parámetro de paginación
base_url = 'https://www.transfermarkt.es/laliga/marktwerte/wettbewerb/ES1'

# Diccionario para almacenar todos los jugadores de todas las páginas
most_values_esp = {}

# Función para scrapear una página específica
def scrape_players_from_url(url):
    players = {}  # Diccionario para almacenar los nombres y URLs de los jugadores
    try:
        # Realiza una solicitud HTTP a la URL
        response = requests.get(url, headers=headers)
        response.raise_for_status()  # Verifica si la solicitud fue exitosa

        # Parsea el contenido HTML de la página
        soup = BeautifulSoup(response.text, 'html.parser')

        # Encuentra la tabla que contiene los datos
        table = soup.find('table', {'class': 'items'})

        if not table:
            print(f"No se encontró la tabla en la URL: {url}. La estructura de la página puede haber cambiado.")
        else:
            # Itera sobre las filas de la tabla
            for row in table.find_all('tr', {'class': ['odd', 'even']}):
                # Encuentra la celda con el nombre del jugador y la URL de su perfil
                name_cell = row.find('td', {'class': 'hauptlink'})
                if name_cell:
                    name = name_cell.text.strip()
                    profile_url = 'https://www.transfermarkt.es' + name_cell.find('a')['href']
                    players[name] = profile_url  # Agrega el jugador al diccionario
    except requests.HTTPError as e:
        print(f'Error HTTP: {e}')
    except Exception as e:
        print(f'Error: {e}')
    
    return players

# Función para determinar el número de páginas a scrapear
def get_total_pages(url):
    try:
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Busca el número de la última página en la paginación
        pagination = soup.find('div', class_='pager')
        if pagination:
            # Busca todos los enlaces de paginación y toma el penúltimo, ya que el último es "Siguiente"
            page_links = pagination.find_all('a')
            last_page_link = page_links[-2] if len(page_links) > 1 else page_links[-1]
            last_page_number = last_page_link.text.strip()
            
            # Asegúrate de que el número es un valor válido
            return int(last_page_number) if last_page_number.isdigit() else 1
        else:
            return 1  # Si no se encuentra la paginación, se asume que solo hay una página
    except Exception as e:
        print(f'Error al obtener el número de páginas: {e}')
        return 1

# Obtiene el número total de páginas
total_pages = get_total_pages(base_url)

# Itera sobre cada página
for page in range(1, total_pages + 1):
    # Cambia el formato de la URL de paginación
    page_url = f'{base_url}/plus/1/galerie/0/page/{page}'
    print(f"Scrapeando página {page}/{total_pages}: {page_url}")
    
    # Scrapea los jugadores de la página actual
    players_on_page = scrape_players_from_url(page_url)
    
    # Agrega los jugadores al diccionario global
    most_values_esp.update(players_on_page)



Scrapeando página 1/1: https://www.transfermarkt.es/laliga/marktwerte/wettbewerb/ES1/plus/1/galerie/0/page/1


In [5]:
most_values_esp

{'Kylian Mbappé': 'https://www.transfermarkt.es/kylian-mbappe/profil/spieler/342229',
 'Jude Bellingham': 'https://www.transfermarkt.es/jude-bellingham/profil/spieler/581678',
 'Vinicius Junior': 'https://www.transfermarkt.es/vinicius-junior/profil/spieler/371998',
 'Federico Valverde': 'https://www.transfermarkt.es/federico-valverde/profil/spieler/369081',
 'Lamine Yamal': 'https://www.transfermarkt.es/lamine-yamal/profil/spieler/937958',
 'Rodrygo': 'https://www.transfermarkt.es/rodrygo/profil/spieler/412363',
 'Aurélien Tchouaméni': 'https://www.transfermarkt.es/aurelien-tchouameni/profil/spieler/413112',
 'Eduardo Camavinga': 'https://www.transfermarkt.es/eduardo-camavinga/profil/spieler/640428',
 'Gavi': 'https://www.transfermarkt.es/gavi/profil/spieler/646740',
 'Julián Alvarez': 'https://www.transfermarkt.es/julian-alvarez/profil/spieler/576024',
 'Pedri': 'https://www.transfermarkt.es/pedri/profil/spieler/683840',
 'Nico Williams': 'https://www.transfermarkt.es/nico-williams/pr

Jugadores mas valiosos

In [16]:
import requests
from bs4 import BeautifulSoup

# Cabeceras HTTP para simular una solicitud desde un navegador
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36'
}

# URL base sin el parámetro de paginación
base_url = 'https://www.transfermarkt.es/spieler-statistik/wertvollstespieler/marktwertetop'

# Diccionario para almacenar todos los jugadores de todas las páginas
most_values = {}

# Función para scrapear una página específica
def scrape_players_from_url(url):
    players = {}  # Diccionario para almacenar los nombres y URLs de los jugadores
    try:
        # Realiza una solicitud HTTP a la URL
        response = requests.get(url, headers=headers)
        response.raise_for_status()  # Verifica si la solicitud fue exitosa

        # Parsea el contenido HTML de la página
        soup = BeautifulSoup(response.text, 'html.parser')

        # Imprime el HTML para depuración
        # print(soup.prettify())

        # Encuentra la tabla que contiene los datos
        table = soup.find('table', {'class': 'items'})
        if not table:
            print(f"No se encontró la tabla en la URL: {url}. La estructura de la página puede haber cambiado.")
            return players
        
        # Itera sobre las filas de la tabla
        for row in table.find_all('tr', {'class': ['odd', 'even']}):
            # Encuentra la celda con el nombre del jugador y la URL de su perfil
            name_cell = row.find('td', {'class': 'hauptlink'})
            if name_cell:
                name = name_cell.text.strip()
                profile_url = 'https://www.transfermarkt.es' + name_cell.find('a')['href']
                players[name] = profile_url  # Agrega el jugador al diccionario
    except requests.HTTPError as e:
        print(f'Error HTTP: {e}')
    except Exception as e:
        print(f'Error: {e}')
    
    return players

# Función para determinar el número de páginas a scrapear
def get_total_pages(url):
    try:
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Imprime el HTML para depuración
        # print(soup.prettify())

        # Busca el número de la última página en la paginación
        pagination = soup.find('div', class_='pager')
        if pagination:
            # Busca todos los enlaces de paginación y toma el penúltimo, ya que el último es "Siguiente"
            page_links = pagination.find_all('a')
            last_page_link = page_links[-2] if len(page_links) > 1 else page_links[-1]
            last_page_number = last_page_link.text.strip()
            
            # Asegúrate de que el número es un valor válido
            return int(last_page_number) if last_page_number.isdigit() else 1
        else:
            return 1  # Si no se encuentra la paginación, se asume que solo hay una página
    except Exception as e:
        print(f'Error al obtener el número de páginas: {e}')
        return 1

# Obtiene el número total de páginas
total_pages = get_total_pages(base_url)

# Itera sobre cada página
for page in range(1, total_pages + 1):
    # Cambia el formato de la URL de paginación
    page_url = f'{base_url}/plus/1/galerie/0/page/{page}'
    print(f"Scrapeando página {page}/{total_pages}: {page_url}")
    
    # Scrapea los jugadores de la página actual
    players_on_page = scrape_players_from_url(page_url)
    
    # Agrega los jugadores al diccionario global
    most_values.update(players_on_page)


Scrapeando página 1/1: https://www.transfermarkt.es/spieler-statistik/wertvollstespieler/marktwertetop/plus/1/galerie/0/page/1


In [17]:
most_values

{'Jude Bellingham': 'https://www.transfermarkt.es/jude-bellingham/profil/spieler/581678',
 'Erling Haaland': 'https://www.transfermarkt.es/erling-haaland/profil/spieler/418560',
 'Vinicius Junior': 'https://www.transfermarkt.es/vinicius-junior/profil/spieler/371998',
 'Kylian Mbappé': 'https://www.transfermarkt.es/kylian-mbappe/profil/spieler/342229',
 'Phil Foden': 'https://www.transfermarkt.es/phil-foden/profil/spieler/406635',
 'Bukayo Saka': 'https://www.transfermarkt.es/bukayo-saka/profil/spieler/433177',
 'Florian Wirtz': 'https://www.transfermarkt.es/florian-wirtz/profil/spieler/598577',
 'Jamal Musiala': 'https://www.transfermarkt.es/jamal-musiala/profil/spieler/580195',
 'Rodri': 'https://www.transfermarkt.es/rodri/profil/spieler/357565',
 'Lamine Yamal': 'https://www.transfermarkt.es/lamine-yamal/profil/spieler/937958',
 'Federico Valverde': 'https://www.transfermarkt.es/federico-valverde/profil/spieler/369081',
 'Declan Rice': 'https://www.transfermarkt.es/declan-rice/profil

PRUEBAS CON FUNCION

In [6]:
# Lista de URLs de los jugadores
jugadores = {
    'Kylian Mbappé': 'https://www.transfermarkt.es/kylian-mbappe/profil/spieler/342229',
    'Jude Bellingham': 'https://www.transfermarkt.es/jude-bellingham/profil/spieler/581678',
    'Vinicius Junior': 'https://www.transfermarkt.es/vinicius-junior/profil/spieler/371998',
    'Lamine Yamal': 'https://www.transfermarkt.es/lamine-yamal/profil/spieler/937958',
    'Federico Valverde': 'https://www.transfermarkt.es/federico-valverde/profil/spieler/369081',
    'Rodrygo': 'https://www.transfermarkt.es/rodrygo/profil/spieler/412363',
    'Eduardo Camavinga': 'https://www.transfermarkt.es/eduardo-camavinga/profil/spieler/640428',
    'Aurélien Tchouaméni': 'https://www.transfermarkt.es/aurelien-tchouameni/profil/spieler/413112',
    'Julián Álvarez': 'https://www.transfermarkt.es/julian-alvarez/profil/spieler/576024',
    'Gavi': 'https://www.transfermarkt.es/gavi/profil/spieler/646740',
    'Pedri': 'https://www.transfermarkt.es/pedri/profil/spieler/683840',
    'Nico Williams': 'https://www.transfermarkt.es/nico-williams/profil/spieler/709187',
    'Frenkie de Jong': 'https://www.transfermarkt.es/frenkie-de-jong/profil/spieler/326330',
    'Ronald Araújo': 'https://www.transfermarkt.es/ronald-araujo/profil/spieler/480267',
    'Éder Militão': 'https://www.transfermarkt.es/eder-militao/profil/spieler/401530',
    'Endrick': 'https://www.transfermarkt.es/endrick/profil/spieler/971570',
    'Dani Olmo': 'https://www.transfermarkt.es/dani-olmo/profil/spieler/293385',
    'Takefusa Kubo': 'https://www.transfermarkt.es/takefusa-kubo/profil/spieler/405398',
    'Martín Zubimendi': 'https://www.transfermarkt.es/martin-zubimendi/profil/spieler/423440',
    'Raphinha': 'https://www.transfermarkt.es/raphinha/profil/spieler/411295',
    'Conor Gallagher': 'https://www.transfermarkt.es/conor-gallagher/profil/spieler/488362',
    'Mikel Oyarzabal': 'https://www.transfermarkt.es/mikel-oyarzabal/profil/spieler/351478',
    'Arda Güler': 'https://www.transfermarkt.es/arda-guler/profil/spieler/861410',
    'Jules Koundé': 'https://www.transfermarkt.es/jules-kounde/profil/spieler/411975',
    'Giorgi Mamardashvili': 'https://www.transfermarkt.es/giorgi-mamardashvili/profil/spieler/502676',
    'Erling Haaland': 'https://www.transfermarkt.es/erling-haaland/profil/spieler/418560',
    'Phil Foden': 'https://www.transfermarkt.es/phil-foden/profil/spieler/406635',
    'Bukayo Saka': 'https://www.transfermarkt.es/bukayo-saka/profil/spieler/433177',
    'Florian Wirtz': 'https://www.transfermarkt.es/florian-wirtz/profil/spieler/598577',
    'Jamal Musiala': 'https://www.transfermarkt.es/jamal-musiala/profil/spieler/580195',
    'Rodri': 'https://www.transfermarkt.es/rodri/profil/spieler/357565',
    'Declan Rice': 'https://www.transfermarkt.es/declan-rice/profil/spieler/357662',
    'Lautaro Martinez': 'https://www.transfermarkt.es/lautaro-martinez/profil/spieler/406625',
    'Martin Odegaard': 'https://www.transfermarkt.es/martin-odegaard/profil/spieler/316264',
    'Victor Osimhen': 'https://www.transfermarkt.es/victor-osimhen/profil/spieler/401923',
    'Harry Kane': 'https://www.transfermarkt.es/harry-kane/profil/spieler/132098',
    'Rafael Leao': 'https://www.transfermarkt.es/rafael-leao/profil/spieler/357164',
    'Bruno Guimaraes': 'https://www.transfermarkt.es/bruno-guimaraes/profil/spieler/520624',
    'Cole Palmer': 'https://www.transfermarkt.es/cole-palmer/profil/spieler/568177'
}

In [7]:
# Función para extraer información de un jugador a partir de su URL
def extraer_info_jugador(url):
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Diccionario para almacenar los datos del jugador
    datos_jugador = {}

    # Extraer el nombre completo
    try:
        headline_wrapper = soup.find('h1', class_='data-header__headline-wrapper')
        numero = headline_wrapper.find('span', class_='data-header__shirt-number').text.strip()
        nombre = headline_wrapper.text.replace(numero, "").strip()
    except AttributeError:
        nombre = "Nombre no encontrado"
    datos_jugador['Nombre'] = nombre

    # Extraer la información del fichaje
    try:
        club_info = soup.find('div', class_='data-header__club-info')
        equipo = club_info.find('span', class_='data-header__club').find('a').text.strip()
        liga = club_info.find('span', class_='data-header__league').text.strip()

        # Verificar si hay suficientes elementos en la lista antes de acceder
        labels = club_info.find_all('span', class_='data-header__label')
        if len(labels) > 1:
            fecha_fichaje = labels[1].find('span', class_='data-header__content').text.strip()
        else:
            fecha_fichaje = "Fecha de fichaje no encontrada"
        
        if len(labels) > 2:
            contrato_hasta = labels[2].find('span', class_='data-header__content').text.strip()
        else:
            contrato_hasta = "Contrato no encontrado"
    except AttributeError:
        equipo = liga = fecha_fichaje = contrato_hasta = "Información no encontrada"
    
    datos_jugador['Equipo'] = equipo
    datos_jugador['Liga'] = liga
    datos_jugador['Fecha Fichaje'] = fecha_fichaje
    datos_jugador['Contrato Hasta'] = contrato_hasta

    # Extraer la fecha de nacimiento y edad
    try:
        birth_date_span = soup.find('span', {'itemprop': 'birthDate'})
        fecha_nacimiento = birth_date_span.get_text(strip=True) if birth_date_span else "Fecha de nacimiento no encontrada"
    except AttributeError:
        fecha_nacimiento = "Fecha de nacimiento no encontrada"
    datos_jugador['Fecha Nacimiento'] = fecha_nacimiento

    # Extraer el lugar de nacimiento
    try:
        birth_place_span = soup.find('span', {'itemprop': 'birthPlace'})
        lugar_nacimiento = birth_place_span.get_text(strip=True) if birth_place_span else "Lugar de nacimiento no encontrado"
    except AttributeError:
        lugar_nacimiento = "Lugar de nacimiento no encontrado"
    datos_jugador['Lugar Nacimiento'] = lugar_nacimiento

    # Extraer la nacionalidad
    try:
        nationality_span = soup.find('span', {'itemprop': 'nationality'})
        nacionalidad = nationality_span.get_text(strip=True).split('\n')[-1].strip() if nationality_span else "Nacionalidad no encontrada"
    except AttributeError:
        nacionalidad = "Nacionalidad no encontrada"
    datos_jugador['Nacionalidad'] = nacionalidad

    # Extraer la altura
    try:
        height_span = soup.find('span', {'itemprop': 'height'})
        altura = height_span.get_text(strip=True) if height_span else "Altura no encontrada"
    except AttributeError:
        altura = "Altura no encontrada"
    datos_jugador['Altura'] = altura

    # Extraer la posición
    try:
        posicion = "Posición no encontrada"
        uls = soup.find_all('ul', class_='data-header__items')
        if len(uls) > 1:
            position_li = next((li for li in uls[1].find_all('li', class_='data-header__label') if 'Posición:' in li.get_text()), None)
            if position_li:
                position_span = position_li.find('span', class_='data-header__content')
                posicion = position_span.get_text(strip=True) if position_span else "Posición no encontrada"
    except AttributeError:
        posicion = "Posición no encontrada"
    datos_jugador['Posición'] = posicion

   # Extraer la selección
    try:
        # Primer intento de extraer la selección
        selection_li = next((li for li in soup.find_all('li', class_='data-header__label') if 'Selección:' in li.get_text()), None)
        if selection_li:
            selection_span = selection_li.find('span', class_='data-header__content')
            if selection_span:
                selection = ' '.join(selection_span.get_text(strip=True).split())
            else:
                selection = "Selección no encontrada"
        else:
            # Si el primer intento falla, se usa el segundo método
            seleccion_li = next((li for li in soup.find_all('li', class_='data-header__label') if 'Jugador de la selección' in li.get_text()), None)
            if seleccion_li:
                seleccion_span = seleccion_li.find('span', class_='data-header__content')
                if seleccion_span:
                    selection = seleccion_span.get_text(strip=True)
                else:
                    selection = "Selección no encontrada"
            else:
                selection = "Selección no encontrada"
    except AttributeError:
        selection = "Selección no encontrada"

    datos_jugador['Selección'] = selection

    # Extraer partidos internacionales y goles
    try:
        stats_li = next((li for li in soup.find_all('li', class_='data-header__label') if 'Partidos internac./goles:' in li.get_text()), None)
        if stats_li:
            stats_links = stats_li.find_all('a', class_='data-header__content--highlight')
            if len(stats_links) == 2:
                partidos = stats_links[0].get_text(strip=True)
                goles = stats_links[1].get_text(strip=True)
            else:
                partidos = "No disponible"
                goles = "No disponible"
        else:
            partidos = goles = "Partidos y goles no encontrados"
    except AttributeError:
        partidos = goles = "Partidos y goles no encontrados"
    datos_jugador['Partidos Internacionales'] = partidos
    datos_jugador['Goles'] = goles

    # Extraer el valor de mercado
    try:
        market_value_div = soup.find('div', class_='data-header__box--small')
        if market_value_div:
            market_value_text = market_value_div.get_text(separator=' ', strip=True)
            market_value = market_value_text.split('Última revisión:')[0].strip()
            last_update = market_value_text.split('Última revisión:')[-1].strip() if 'Última revisión:' in market_value_text else "Última revisión no encontrada"
        else:
            market_value = "Valor de mercado no encontrado"
            last_update = "Última revisión no encontrada"
    except AttributeError:
        market_value = "Valor de mercado no encontrado"
        last_update = "Última revisión no encontrada"
    datos_jugador['Valor Mercado'] = market_value
    datos_jugador['Última Revisión'] = last_update

    # Extraer el pie
    try:
        pie_li = soup.find('span', string='Pie:')
        pie_span = pie_li.find_next_sibling('span', class_='info-table__content--bold') if pie_li else None
        pie = pie_span.get_text(strip=True) if pie_span else "Pie no encontrado"
    except AttributeError:
        pie = "Pie no encontrado"
    datos_jugador['Pie'] = pie

    # Extraer el agente
    try:
        agente_li = soup.find('span', string='Agente:')
        agente_span = agente_li.find_next_sibling('span', class_='info-table__content--bold') if agente_li else None
        if agente_span:
            agente = agente_span.get_text(strip=True)
            if "Miembro de su ..." in agente:
                agente = "Miembro de su familia"
        else:
            agente = "Agente no encontrado"
    except AttributeError:
        agente = "Agente no encontrado"
    datos_jugador['Agente'] = agente

    # Extraer el proveedor
    try:
        proveedor_li = soup.find('span', string='Proveedor:')
        proveedor_span = proveedor_li.find_next_sibling('span', class_='info-table__content--bold') if proveedor_li else None
        proveedor = proveedor_span.get_text(strip=True) if proveedor_span else "Proveedor no encontrado"
    except AttributeError:
        proveedor = "Proveedor no encontrado"
    datos_jugador['Proveedor'] = proveedor

    return datos_jugador

In [8]:
# Lista para almacenar la información de todos los jugadores
lista_jugadores = []

# Iterar sobre el diccionario de URLs y extraer la información
for nombre_jugador, url_jugador in jugadores.items():
    #print(f"Extrayendo información de {nombre_jugador}...")
    info = extraer_info_jugador(url_jugador)
    lista_jugadores.append(info)


In [9]:
# Convertir la lista de diccionarios en un DataFrame
df_jugadores = pd.DataFrame(lista_jugadores)
df_jugadores

,Nombre,Equipo,Liga,Fecha Fichaje,Contrato Hasta,Fecha Nacimiento,Lugar Nacimiento,Nacionalidad,Altura,Posición,Selección,Partidos Internacionales,Goles,Valor Mercado,Última Revisión,Pie,Agente,Proveedor
0,Kylian Mbappé,Real Madrid,LaLiga,01/07/2024,30/06/2029,20/12/1998 (25),Paris,Francia,"1,78 m",Delantero centro,Francia,86,48,"180,00 mill. €",03/06/2024,derecho,Miembro de su familia,Nike
1,Jude Bellingham,Real Madrid,LaLiga,01/07/2023,30/06/2029,29/06/2003 (21),Stourbridge,Inglaterra,"1,86 m",Mediocentro ofensivo,Inglaterra,36,5,"180,00 mill. €",07/06/2024,derecho,Agente no encontrado,adidas
2,Vinicius Junior,Real Madrid,LaLiga,12/07/2018,30/06/2027,12/07/2000 (24),São Gonçalo,Brasil,"1,76 m",Extremo izquierdo,Brasil,35,5,"180,00 mill. €",07/06/2024,derecho,Roc Nation Sports,Nike
3,Lamine Yamal,FC Barcelona,LaLiga,Fecha de fichaje no encontrada,Contrato no encontrado,13/07/2007 (17),Lugar de nacimiento no encontrado,España,"1,80 m",Extremo derecho,España,Partidos y goles no encontrados,Partidos y goles no encontrados,"120,00 mill. €",18/07/2024,izquierdo,Agente es conocido / jugador menor de 18 años,adidas
4,Federico Valverde,Real Madrid,LaLiga,01/07/2018,30/06/2029,22/07/1998 (26),Montevideo,Uruguay,"1,82 m",Mediocentro,Uruguay,63,7,"120,00 mill. €",07/06/2024,derecho,Wasserman,Nike
5,Rodrygo,Real Madrid,LaLiga,01/07/2019,30/06/2028,09/01/2001 (23),Osasco,Brasil,"1,74 m",Extremo derecho,Brasil,29,7,"110,00 mill. €",07/06/2024,derecho,Un1que Football,Nike
6,Eduardo Camavinga,Real Madrid,LaLiga,31/08/2021,30/06/2029,10/11/2002 (21),Cabinda,Francia,"1,82 m",Mediocentro,Francia,21,1,"100,00 mill. €",07/06/2024,izquierdo,CAA Stellar,Nike
7,Aurélien Tchouaméni,Real Madrid,LaLiga,01/07/2022,30/06/2028,27/01/2000 (24),Rouen,Francia,"1,88 m",Pivote,Francia,36,3,"100,00 mill. €",07/06/2024,derecho,ESN,Nike
8,Julián Alvarez,Atlético Madrid,LaLiga,Fecha de fichaje no encontrada,Contrato no encontrado,31/01/2000 (24),Lugar de nacimiento no encontrado,Argentina,"1,70 m",Delantero centro,Argentina,Partidos y goles no encontrados,Partidos y goles no encontrados,"90,00 mill. €",27/05/2024,derecho,Fernando Hidalgo,adidas
9,Gavi,FC Barcelona,LaLiga,01/07/2021,30/06/2026,05/08/2004 (20),Los Palacios y ...,España,"1,73 m",Mediocentro,España,27,5,"90,00 mill. €",07/06/2024,derecho,De la Peña & ...,Nike


FOTMOB

In [2]:
import requests
from bs4 import BeautifulSoup

def obtener_datos_jugador(url_jugador):
    # Definir headers para simular un navegador real
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) '
                      'AppleWebKit/537.36 (KHTML, like Gecko) '
                      'Chrome/112.0.0.0 Safari/537.36'
    }

    try:
        # Realizar la solicitud HTTP a la página del jugador
        response = requests.get(url_jugador, headers=headers)
        response.raise_for_status()  # Verificar si la solicitud fue exitosa
    except requests.exceptions.RequestException as e:
        print(f"Error al realizar la solicitud HTTP: {e}")
        return None

    # Crear un objeto BeautifulSoup para analizar el HTML
    soup = BeautifulSoup(response.text, 'html.parser')

    # Extraer el nombre del jugador
    nombre_jugador_tag = soup.find('h1', class_='css-zt63wq-PlayerNameCSS e1i691m81')
    nombre_jugador = nombre_jugador_tag.get_text(strip=True) if nombre_jugador_tag else None

    # Extraer el nombre del equipo
    nombre_equipo_tag = soup.find('div', class_='css-14k6s2u-TeamCSS e1i691m82')
    nombre_equipo = nombre_equipo_tag.get_text(strip=True) if nombre_equipo_tag else None

    # Retornar los datos en un diccionario
    datos_jugador = {
        'nombre_jugador': nombre_jugador,
        'nombre_equipo': nombre_equipo
    }

    return datos_jugador

if __name__ == "__main__":
    # URL del jugador
    url = "https://www.fotmob.com/es/players/1467236"

    # Obtener los datos del jugador
    datos = obtener_datos_jugador(url)

    # Mostrar los datos obtenidos
    if datos:
        print("Datos del Jugador:")
        print(f"Nombre: {datos['nombre_jugador']}")
        print(f"Equipo: {datos['nombre_equipo']}")


Datos del Jugador:
Nombre: Lamine Yamal
Equipo: Barcelona


Datos generales

In [5]:
from bs4 import BeautifulSoup

html = '''
<!-- Aquí incluirías tu HTML completo o el que hayas obtenido -->
<div class="css-1u6v53x-PlayerBioStatCSS e1pz3ljv2"><div class="css-to3w1c-StatValueCSS e1pz3ljv4"><span>180 cm</span></div><div class="css-10h4hmz-StatTitleCSS e1pz3ljv3"><span>Altura</span></div></div>
<div class="css-1u6v53x-PlayerBioStatCSS e1pz3ljv2"><div class="css-to3w1c-StatValueCSS e1pz3ljv4"><span>19</span></div><div class="css-10h4hmz-StatTitleCSS e1pz3ljv3"><span>Camiseta</span></div></div>
<div class="css-1u6v53x-PlayerBioStatCSS e1pz3ljv2"><div class="css-to3w1c-StatValueCSS e1pz3ljv4"><span>17 años</span></div><div class="css-10h4hmz-StatTitleCSS e1pz3ljv3"><span>13 jul 2007</span></div></div>
<div class="css-1u6v53x-PlayerBioStatCSS e1pz3ljv2"><div class="css-to3w1c-StatValueCSS e1pz3ljv4"><span>Izquierdo</span></div><div class="css-10h4hmz-StatTitleCSS e1pz3ljv3"><span>Pie preferido</span></div></div>
<div class="css-1spuxeo-PlayerBioStatCSS e1pz3ljv2"><div class="css-to3w1c-StatValueCSS e1pz3ljv4"><img alt="" src="https://www.fotmob.com/_next/image?url=https%3A%2F%2Fimages.fotmob.com%2Fimage_resources%2Flogo%2Fteamlogo%2Fesp.png&amp;w=32&amp;q=75">España</div><div class="css-10h4hmz-StatTitleCSS e1pz3ljv3"><span>País</span></div></div>
<div class="css-1spuxeo-PlayerBioStatCSS e1pz3ljv2"><div class="css-to3w1c-StatValueCSS e1pz3ljv4"><span>95&nbsp;M€</span></div><div class="css-10h4hmz-StatTitleCSS e1pz3ljv3"><span>Valor de mercado</span></div></div>
'''

# Cargar el HTML con BeautifulSoup
soup = BeautifulSoup(html, 'html.parser')

# Encontrar todas las estadísticas
stats = soup.find_all('div', class_='css-1u6v53x-PlayerBioStatCSS')

# Iterar sobre cada estadística
for stat in stats:
    # Extraer el valor
    value = stat.find('div', class_='css-to3w1c-StatValueCSS').get_text(strip=True)
    # Extraer el tipo de estadística
    stat_type = stat.find('div', class_='css-10h4hmz-StatTitleCSS').get_text(strip=True)
    print(f'{stat_type}: {value}')


Altura: 180 cm
Camiseta: 19
13 jul 2007: 17 años
Pie preferido: Izquierdo


In [9]:
from bs4 import BeautifulSoup

html = '''
<!-- Aquí iría el HTML que contiene los datos -->
<div class="css-1kkujsn-StatBox e1ahduwc6"><div class="css-170fd60-StatValue e1ahduwc5"><span>3</span></div><span class="css-1m9s03m-StatTitle e1ahduwc4">Goles</span></div>
<div class="css-1kkujsn-StatBox e1ahduwc6"><div class="css-170fd60-StatValue e1ahduwc5"><span>4</span></div><span class="css-1m9s03m-StatTitle e1ahduwc4">Asistencias</span></div>
<div class="css-1kkujsn-StatBox e1ahduwc6"><div class="css-170fd60-StatValue e1ahduwc5"><span>5</span></div><span class="css-1m9s03m-StatTitle e1ahduwc4">Comenzado</span></div>
<div class="css-1kkujsn-StatBox e1ahduwc6"><div class="css-170fd60-StatValue e1ahduwc5"><span>5</span></div><span class="css-1m9s03m-StatTitle e1ahduwc4">Partidos</span></div>
<div class="css-1kkujsn-StatBox e1ahduwc6"><div class="css-170fd60-StatValue e1ahduwc5"><span>443</span></div><span class="css-1m9s03m-StatTitle e1ahduwc4">Minutos jugados</span></div>
<div class="css-1kkujsn-StatBox e1ahduwc6"><div class="css-1viv3lr-PlayerRatingStyled e1v8brex0"><span>8,66</span></div><span class="css-1m9s03m-StatTitle e1ahduwc4">Puntuación</span></div>
<div class="css-1bw13og-StatBox e1ahduwc6"><div class="css-170fd60-StatValue e1ahduwc5"><svg xmlns="http://www.w3.org/2000/svg" width="20" height="20" viewBox="0 0 20 20"><g id="prefix__ic_yellow_card" transform="translate(-700 -508)"><rect id="prefix__Rectangle_2316" width="11.111" height="15.556" fill="#ffce2c" data-name="Rectangle 2316" rx="2.5" transform="translate(704.444 510.222)"></rect><path id="prefix__Rectangle_2688" fill="none" d="M0 0H20V20H0z" data-name="Rectangle 2688" transform="translate(700 508)"></path></g></svg><span>1</span></div><span class="css-1m9s03m-StatTitle e1ahduwc4">Tarjetas amarillas</span></div>
<div class="css-1bw13og-StatBox e1ahduwc6"><div class="css-170fd60-StatValue e1ahduwc5"><svg xmlns="http://www.w3.org/2000/svg" width="20" height="20" viewBox="0 0 20 20"><g id="prefix__ic_red_card" transform="translate(-700 -508)"><rect id="prefix__Rectangle_2316" width="11.111" height="15.556" fill="#e55e5b" data-name="Rectangle 2316" rx="2.5" transform="translate(704.444 510.222)"></rect><path id="prefix__Rectangle_2688" fill="none" d="M0 0H20V20H0z" data-name="Rectangle 2688" transform="translate(700 508)"></path></g></svg><span>0</span></div><span class="css-1m9s03m-StatTitle e1ahduwc4">Tarjetas rojas</span></div>
'''

# Cargar el HTML con BeautifulSoup
soup = BeautifulSoup(html, 'html.parser')

# Encontrar todas las estadísticas
stats = soup.find_all('div', class_='css-1kkujsn-StatBox')

# Iterar sobre cada estadística
for stat in stats:
    # Intentar extraer el valor
    value_div = stat.find('div', class_='css-170fd60-StatValue')
    if value_div:
        value = value_div.get_text(strip=True)
    else:
        # Para la puntuación
        value_div = stat.find('div', class_='css-1viv3lr-PlayerRatingStyled')
        value = value_div.get_text(strip=True) if value_div else 'N/A'

    # Extraer el tipo de estadística
    stat_type = stat.find('span', class_='css-1m9s03m-StatTitle').get_text(strip=True)
    print(f'{stat_type}: {value}')

# También extraer tarjetas que tienen una estructura diferente
cards = soup.find_all('div', class_='css-1bw13og-StatBox')
for card in cards:
    value_div = card.find('span')  # Obtener el valor del span directamente
    value = value_div.get_text(strip=True) if value_div else 'N/A'
    stat_type = card.find('span', class_='css-1m9s03m-StatTitle').get_text(strip=True)
    print(f'{stat_type}: {value}')


Goles: 3
Asistencias: 4
Comenzado: 5
Partidos: 5
Minutos jugados: 443
Puntuación: 8,66
Tarjetas amarillas: 1
Tarjetas rojas: 0


In [11]:
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime

# Primer bloque de HTML
html_bio = '''
<div class="css-1u6v53x-PlayerBioStatCSS e1pz3ljv2"><div class="css-to3w1c-StatValueCSS e1pz3ljv4"><span>180 cm</span></div><div class="css-10h4hmz-StatTitleCSS e1pz3ljv3"><span>Altura</span></div></div>
<div class="css-1u6v53x-PlayerBioStatCSS e1pz3ljv2"><div class="css-to3w1c-StatValueCSS e1pz3ljv4"><span>19</span></div><div class="css-10h4hmz-StatTitleCSS e1pz3ljv3"><span>Camiseta</span></div></div>
<div class="css-1u6v53x-PlayerBioStatCSS e1pz3ljv2"><div class="css-to3w1c-StatValueCSS e1pz3ljv4"><span>17 años</span></div><div class="css-10h4hmz-StatTitleCSS e1pz3ljv3"><span>13 jul 2007</span></div></div>
<div class="css-1u6v53x-PlayerBioStatCSS e1pz3ljv2"><div class="css-to3w1c-StatValueCSS e1pz3ljv4"><span>Izquierdo</span></div><div class="css-10h4hmz-StatTitleCSS e1pz3ljv3"><span>Pie preferido</span></div></div>
<div class="css-1spuxeo-PlayerBioStatCSS e1pz3ljv2"><div class="css-to3w1c-StatValueCSS e1pz3ljv4"><img alt="" src="https://www.fotmob.com/_next/image?url=https%3A%2F%2Fimages.fotmob.com%2Fimage_resources%2Flogo%2Fteamlogo%2Fesp.png&amp;w=32&amp;q=75">España</div><div class="css-10h4hmz-StatTitleCSS e1pz3ljv3"><span>País</span></div></div>
<div class="css-1spuxeo-PlayerBioStatCSS e1pz3ljv2"><div class="css-to3w1c-StatValueCSS e1pz3ljv4"><span>95&nbsp;M€</span></div><div class="css-10h4hmz-StatTitleCSS e1pz3ljv3"><span>Valor de mercado</span></div></div>
'''

# Segundo bloque de HTML
html_stats = '''
<div class="css-1kkujsn-StatBox e1ahduwc6"><div class="css-170fd60-StatValue e1ahduwc5"><span>3</span></div><span class="css-1m9s03m-StatTitle e1ahduwc4">Goles</span></div>
<div class="css-1kkujsn-StatBox e1ahduwc6"><div class="css-170fd60-StatValue e1ahduwc5"><span>4</span></div><span class="css-1m9s03m-StatTitle e1ahduwc4">Asistencias</span></div>
<div class="css-1kkujsn-StatBox e1ahduwc6"><div class="css-170fd60-StatValue e1ahduwc5"><span>5</span></div><span class="css-1m9s03m-StatTitle e1ahduwc4">Comenzado</span></div>
<div class="css-1kkujsn-StatBox e1ahduwc6"><div class="css-170fd60-StatValue e1ahduwc5"><span>5</span></div><span class="css-1m9s03m-StatTitle e1ahduwc4">Partidos</span></div>
<div class="css-1kkujsn-StatBox e1ahduwc6"><div class="css-170fd60-StatValue e1ahduwc5"><span>443</span></div><span class="css-1m9s03m-StatTitle e1ahduwc4">Minutos jugados</span></div>
<div class="css-1kkujsn-StatBox e1ahduwc6"><div class="css-1viv3lr-PlayerRatingStyled e1v8brex0"><span>8,66</span></div><span class="css-1m9s03m-StatTitle e1ahduwc4">Puntuación</span></div>
<div class="css-1bw13og-StatBox e1ahduwc6"><div class="css-170fd60-StatValue e1ahduwc5"><svg xmlns="http://www.w3.org/2000/svg" width="20" height="20" viewBox="0 0 20 20"><g id="prefix__ic_yellow_card" transform="translate(-700 -508)"><rect id="prefix__Rectangle_2316" width="11.111" height="15.556" fill="#ffce2c" data-name="Rectangle 2316" rx="2.5" transform="translate(704.444 510.222)"></rect><path id="prefix__Rectangle_2688" fill="none" d="M0 0H20V20H0z" data-name="Rectangle 2688" transform="translate(700 508)"></path></g></svg><span>1</span></div><span class="css-1m9s03m-StatTitle e1ahduwc4">Tarjetas amarillas</span></div>
<div class="css-1bw13og-StatBox e1ahduwc6"><div class="css-170fd60-StatValue e1ahduwc5"><svg xmlns="http://www.w3.org/2000/svg" width="20" height="20" viewBox="0 0 20 20"><g id="prefix__ic_red_card" transform="translate(-700 -508)"><rect id="prefix__Rectangle_2316" width="11.111" height="15.556" fill="#e55e5b" data-name="Rectangle 2316" rx="2.5" transform="translate(704.444 510.222)"></rect><path id="prefix__Rectangle_2688" fill="none" d="M0 0H20V20H0z" data-name="Rectangle 2688" transform="translate(700 508)"></path></g></svg><span>0</span></div><span class="css-1m9s03m-StatTitle e1ahduwc4">Tarjetas rojas</span></div>
'''

# Función para extraer datos de bio
def extract_bio_data(html):
    soup = BeautifulSoup(html, 'html.parser')
    bio_data = {}
    stats = soup.find_all('div', class_='css-1u6v53x-PlayerBioStatCSS')
    for stat in stats:
        value = stat.find('div', class_='css-to3w1c-StatValueCSS').get_text(strip=True)
        stat_type = stat.find('div', class_='css-10h4hmz-StatTitleCSS').get_text(strip=True)
        bio_data[stat_type] = value
    return bio_data

# Función para extraer datos de estadísticas
def extract_stat_data(html):
    soup = BeautifulSoup(html, 'html.parser')
    stat_data = {}
    stats = soup.find_all('div', class_='css-1kkujsn-StatBox')
    for stat in stats:
        value_div = stat.find('div', class_='css-170fd60-StatValue')
        if value_div:
            value = value_div.get_text(strip=True)
        else:
            value_div = stat.find('div', class_='css-1viv3lr-PlayerRatingStyled')
            value = value_div.get_text(strip=True) if value_div else 'N/A'
        stat_type = stat.find('span', class_='css-1m9s03m-StatTitle').get_text(strip=True)
        stat_data[stat_type] = value

    # Extraer tarjetas
    cards = soup.find_all('div', class_='css-1bw13og-StatBox')
    for card in cards:
        value_div = card.find('span')  # Obtener el valor del span directamente
        value = value_div.get_text(strip=True) if value_div else 'N/A'
        stat_type = card.find('span', class_='css-1m9s03m-StatTitle').get_text(strip=True)
        stat_data[stat_type] = value

    return stat_data

# Extraer datos
bio_data = extract_bio_data(html_bio)
stat_data = extract_stat_data(html_stats)

# Calcular la edad a partir de la fecha de nacimiento
dob_str = "13 jul 2007"
dob = datetime.strptime(dob_str, "%d %b %Y")
age = (datetime.now() - dob).days // 365  # Calcular la edad en años
bio_data['Edad'] = age  # Actualizar el diccionario con la edad

# Eliminar la entrada de fecha de nacimiento
bio_data.pop("13 jul 2007", None)

# Combinar los datos
combined_data = {**bio_data, **stat_data}

# Crear DataFrame
df = pd.DataFrame(list(combined_data.items()), columns=['Estadística', 'Valor'])

# Mostrar el DataFrame
print(df)


           Estadística      Valor
0               Altura     180 cm
1             Camiseta         19
2        Pie preferido  Izquierdo
3                 Edad         17
4                Goles          3
5          Asistencias          4
6            Comenzado          5
7             Partidos          5
8      Minutos jugados        443
9           Puntuación       8,66
10  Tarjetas amarillas          1
11      Tarjetas rojas          0
